In [6]:
#import the needed libraries
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [7]:
#create the SparkSession
spark = (SparkSession
        .builder
        .appName("Spark Ejs")
        .getOrCreate())

## Exercises from Chapter 2

In [8]:
#a

#route of the file to read
route = "Desktop\Big Data\el_quijote.txt"

#loads the file into data
#Spark supports csv, json and text files(txt) 
data = (spark.read.format("text")
        .option("header", "false")
        .load(route))
    
data.count() #2186
#data.show()  #shows the top 20 rows in tabular form
#data.show(5) #shows the selected number of rows in tabular form
#data.show(truncate = False) #shows the whole text (doesnt put "...", shows all)
#data.show(5, truncate = False) #shows the selected number of rows
#data.show(5, truncate = False, vertical = False) #shows the selected number of rows, truncated to 25 character in horizontal
#in Python is allowed to put False when expected int because I think it internally cast it to int (False is 0)

#data.head()  #returns the first row
#data.first() #returns the first row
#data.head(5) #takes the number selected of rows from the start and puts them in an array 
#data.take(5) #takes the number selected of rows from the start and puts them in an array (works as head(<n>) does) 
#data.tail(5) #takes the number selected of rows from the end and puts them in an array

#There is no diference in the behaviour between head and first, the only difference is that head can take parametres and first can't
#head(<n>) and take(<n>) return the same thing



2186

In [31]:
#b

route = "Desktop/Big Data/mnm_dataset.csv.txt"

data = spark.read.format("csv") \
    .option("header", "true") \
    .load(route)

# totalByState = (data.select("State", "Color", "Count")
#     .groupBy("State", "Color")
#     .agg(sum("Count").alias("Total")))

# totalByState.show()

# totalByState.select("State", "Color", "Total") \
#     .groupBy("State") \
#     .agg(max("Total").alias("Max")) \
#     .orderBy(desc("State")).show()


# caCount = (data.select("State", "Color", "Count")
#     .where(col("State") == "CA")
#     .groupBy("State", "Color")
#     .agg(sum("Count").alias("Total"))
#     .orderBy(desc("Total")).show())

# data.select("State", "Color", "Count") \
#     .where((col("State") == "NV") | (col("State") == "TX") | (col("State") == "CA") | (col("State") == "CO")) \
#     .groupBy("State", "Color") \
#     .agg(sum("Count").alias("Total")) \
#     .orderBy(asc("State"),asc("Color")) \
#     .show()

data.select("State", "Color", "Count") \
    .groupBy("State", "Color") \
    .agg(max("Count").alias("Max"), avg("Count").alias("Average")) \
    .orderBy(asc("State"), asc("Color")) \
    .show()
    

+-----+------+---+------------------+
|State| Color|Max|           Average|
+-----+------+---+------------------+
|   AZ|  Blue| 99| 54.99449877750611|
|   AZ| Brown| 99|54.350412249705535|
|   AZ| Green| 99| 54.82219570405728|
|   AZ|Orange| 99| 54.28300769686205|
|   AZ|   Red| 99|54.637135922330096|
|   AZ|Yellow| 99| 54.98548972188634|
|   CA|  Blue| 99| 55.59762944479102|
|   CA| Brown| 99|55.740395809080326|
|   CA| Green| 99|54.268717353453276|
|   CA|Orange| 99|54.502715751357876|
|   CA|   Red| 99| 55.26992753623188|
|   CA|Yellow| 99|  55.8693967902601|
|   CO|  Blue| 99| 55.11032448377581|
|   CO| Brown| 99| 56.57729468599034|
|   CO| Green| 99| 54.71336835960304|
|   CO|Orange| 99|55.402557856272836|
|   CO|   Red| 99|55.089285714285715|
|   CO|Yellow| 99| 55.22254503195816|
|   NM|  Blue| 99| 55.03663003663004|
|   NM| Brown| 99|55.392412566686424|
+-----+------+---+------------------+
only showing top 20 rows

